In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import resnet50
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import pandas as pd
import numpy as np
import cv2
import os

In [37]:
# Globals   
path_to_data_dir = "../../database/sorted_by_state/"
test_file = "Alabama/_BK45Fzhx9-uhaBc8M77Eg.jpeg"

In [38]:
cv2.imread(path_to_data_dir + test_file).shape
train = ImageDataGenerator(rescale=1./255)
validation = ImageDataGenerator(rescale=1./255)

In [39]:
train_dataset = train.flow_from_directory(path_to_data_dir, target_size=(224, 224), batch_size=64, class_mode='categorical', shuffle=True)
validation_dataset = train.flow_from_directory(path_to_data_dir, target_size=(224, 224), batch_size=64, class_mode='categorical', shuffle=True)

Found 51175 images belonging to 54 classes.
Found 51175 images belonging to 54 classes.


In [40]:
# model with 3 convolution layers max pooled 3 times with relu as activation function and softmax applied to output layer
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', use_bias=True),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu', use_bias=True),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [43]:
model_fit = model.fit(train_dataset, epochs=2, validation_data=validation_dataset, verbose=1)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.